# **FLOW ACCUMULATION**

## Loading the required libraries

**This section of code loads the libraries needed to create the FLOW ACCUMULATION script. Here is a brief description of each of the libraries used:**

- `numpy`: for working with numeric data and vector operations
- `pysheds.grid`: for handling digital elevation model (DEM) and hydrological analyses
- `matplotlib.pyplot`: for data visualisation and graphical output
- `matplotlib.colors`: for working with colours
- `matplotlib.patches`: for creating graphical elements
- `seaborn`: for statistical data visualization, extension over matplotlib
- `rasterio`: for working with rasters and reading/writing them

In [1]:
# Loading the necessary libraries
import numpy as np
from pysheds.grid import Grid
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patches as mpatches
import seaborn as sns
import rasterio

## Loading digital elevation model (DEM)

**This part of the code retrieves the digital terrain model (DEM) from the given path.** 

- The `dem_file` variable contains the path to the DEM file.
- The `grid` variable creates a Grid object from the `pysheds` library that will be used to manipulate the DEM data. 
- The `dem` variable contains the actual DEM data read from the file using the `grid` object.

In [ ]:
# Insert relative path to file
path='data/dem3.tif'

# Load digital elevation model (DEM) from the attached path
grid = Grid.from_raster(path)
dem = grid.read_raster(path)
dem_file = path

## Creating a digital terrain model (DEM) preview

**This section of code creates a visual preview of the digital terrain model. The graph shows the elevations of different areas.**

- The graph has a defined size and a transparent background, which makes it easier to read.
- The 'terrain' color scheme is used to display the different elevation levels. In this way, different altitudes can be easily distinguished.
- A legend is added to the chart to allow the user to easily identify the elevation values displayed on the chart.

In [ ]:
# Create a preview and axis chart and transparency
fig, ax = plt.subplots(figsize=(10,6))
fig.patch.set_alpha(0)

# Display DEM using the 'terrain' color scheme
plt.imshow(dem, extent=grid.extent, cmap='terrain', zorder=1)

# Adding a legend
plt.colorbar(label='Elevation (m)')

# Adding a grid
plt.grid(zorder=0)

# Labels
plt.title('Digital elevation map', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

## Modifications in the Digital Terrain Model (DEM)

**This section of code makes several modifications to the Digital Elevation Model (DEM) to ensure its accuracy and consistency:**

- **Fill potholes**: The `grid.fill_pits()` function fills potholes in the digital elevation model (DEM). Potholes are areas where water flows into a single point, which can cause problems when analyzing hydrologic data.

- **Fill Depressions**: The `grid.fill_depressions()` function fills depressions in a digital elevation model (DEM). Depressions are areas where water does not drain into surrounding areas, which can lead to inconsistent results when analyzing water flow.

- **Flat Area Correction**: The `grid.resolve_flats()` function corrects flat areas in the Digital Elevation Model (DEM). Flat areas are locations where water does not have a clear direction of flow, which can lead to inconsistent results when analyzing water flow. Correcting these areas will ensure that water flow is correctly modeled and analyzed.

In [4]:
# Filling potholes in the digital terrain model
pit_filled_dem = grid.fill_pits(dem)

# Fill depressions
flooded_dem = grid.fill_depressions(pit_filled_dem)

# Fix flat areas in the digital terrain model
inflated_dem = grid.resolve_flats(flooded_dem)

## Definition of flow directions

**This part of the code defines the flow directions that will be used to calculate the water flow in the digital terrain model. The flow directions are defined using numerical values that correspond to the eight cardinal directions.**

- The variable `dirmap` contains a list of eight numeric values that represent the flow directions. The numbers are expressed as powers of 2, allowing easy comparison of flow directions using a binary representation.

- The calculation of flow directions is performed using the `grid.flowdir()` method, which assigns each cell in the digital elevation model (DEM) a flow direction based on the topography of the terrain and the defined directions. In this way, a new `fdir` object is created that contains information about the flow directions throughout the digital terrain model.

In [5]:
# Definice směrů toku (flow directions)
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)

# Výpočet směrů toku na vyplněném DEM
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)

## Plotting the flow direction network

**This part of the code shows the flow direction network that was calculated based on the digital terrain model.**

- The first part of the code creates a graph with a defined size using `plt.figure(figsize=(10,6))`. The background transparency is set to 0 using `fig.patch.set_alpha(0)`, which ensures that the background of the graph will be transparent.
- The flow direction network itself is displayed using `plt.imshow()`. The color scheme `viridis` is used to visualize the different flow directions.
- The legend shows the different flow directions based on the values defined in `dirmap`.
- The axis labels and the chart title are set for better orientation.
- The grid is displayed below the flow direction grid data to help with orientation on the graph.

In [ ]:
# Canvas definition and transparency settings
fig = plt.figure(figsize=(10,6))
fig.patch.set_alpha(0)

# Display flow directions
plt.imshow(fdir, extent=grid.extent, cmap='viridis', zorder=2)

# Legend
boundaries = ([0] + sorted(list(dirmap)))
plt.colorbar(boundaries=boundaries, values=sorted(dirmap))

# Labels
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow direction network', size=14)
plt.grid(zorder=-1)
plt.tight_layout()

## Flow accumulation calculation

**This part of the code calculates the flow accumulation based on the flow directions in the Digital Elevation Model (DEM).**

- The `grid.accumulation()` function calculates the flow accumulation based on the flow directions defined in `fdir`. This flow accumulation determines how much water eventually reaches each cell in the digital terrain model based on the terrain topography and flow directions. 
- The `dirmap` argument is used to define the flow directions that are used in the flow accumulation calculation. This argument ensures consistency between the flow directions used in the calculation of the flow directions and the flow accumulation.

In [7]:
# Flow accumulation calculation based on flow directions
acc = grid.accumulation(fdir, dirmap=dirmap)

## Flow accumulation visualization

**This part of the code visualizes the flow accumulation based on the calculated flow directions.**

- The first part of the code creates a graph with a defined size using `plt.subplots(figsize=(8,6))`. The background transparency is set to 0 using `fig.patch.set_alpha(0)`, which ensures that the graph background is transparent.
- A grid is added to the graph background using `plt.grid('on', zorder=0)`, which helps with orientation on the graph.
- Flow accumulation is shown using `ax.imshow()`. The color scheme `cubehelix` is used to visualize different levels of flux accumulation.
- A legend is added using `plt.colorbar()` with the label 'Cells on top of flux', allowing the user to easily interpret the flux accumulation values.
- The axis labels and the graph caption are set for better orientation.

In [ ]:
# Canvas and transparency settings
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

# Add grid with background
plt.grid('on', zorder=0)

# Display flow accumulation
im = ax.imshow(acc, extent=grid.extent, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, acc.max()),
               interpolation='bilinear')

# Legend + description
plt.colorbar(im, ax=ax, label='Upstream cells')

# Labels
plt.title('Flow accumulation', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

## Finding the lowest point in the digital terrain model

**This section of code works on the digital model to obtain the coordinates of the lowest point in the raster**

- The DEM file is opened using the `rasterio.open(dem_file)` function. 
- The DEM data is read from the file using `src.read(1)` and stored in the variable `dem_data`.
- The lowest elevation value in the digital terrain model is obtained using `dem_data.min()` and stored in the variable `min_elevation`.
- The index of the lowest elevation value is obtained by `dem_data.argmin()` and stored in the variable `min_index`.
- The geographic coordinates corresponding to the minimum elevation value are obtained using `src.xy(min_index[0], min_index[1])` and stored in the variables `x_min` and `y_min`.

In [9]:
# Otevření DEM souboru
with rasterio.open(dem_file) as src:
    # Načtení dat DEM
    dem_data = src.read(1)
    
    # Zjištění minimální hodnoty nadmořské výšky
    min_elevation = dem_data.min()

# Zjištění indexu minimální hodnoty nadmořské výšky
min_index = np.unravel_index(dem_data.argmin(), dem_data.shape)

# Převod indexu na zeměpisné souřadnice
x_min, y_min = src.xy(min_index[0], min_index[1])

## Create a basin for the selected point

- For the selected point, which is defined as the lowest point in the grid, the nearest points on the surface with flux accumulation above 1000 cells are determined using the `grid.snap_to_mask()` function. These points are stored in the variables `x_snap` and `y_snap`.
- A catchment is created for the selected point (determined by `x_snap` and `y_snap`) using the `grid.catchment()` function. This function creates a catchment based on the flow directions in a given digital elevation model (DEM).
- The grid is clipped to the extent of the catchment area using the `grid.clip_to()` function, ensuring that the graphical display contains only the catchment area.

In [10]:
# Select the coordinates of the lowest point in the grid
x_min, y_min

# Determine the nearest point on the surface with flux accumulation above 1000 cells
x_snap, y_snap = grid.snap_to_mask(acc > 1000, (x_min, y_min))

# Create a basin for the selected point (x_snap, y_snap)
catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap, 
                       xytype='coordinate')

# Crop grid to catchment range
grid.clip_to(catch)

# Display the clipped watershed
clipped_catch = grid.view(catch)

## River network extraction

**This section of code extracts the river network based on flow directions and accumulation.**

- The `grid.extract_river_network()` function extracts river networks based on the flow directions in `fdir` and the condition that the flow accumulation must be greater than 50. In this way, river networks are identified based on the topography of the terrain and the amount of water in the flow directions.
- The result of river network extraction is stored in the variable `branches`. This variable contains information about each branch of the river network, including the coordinates and geometry of each branch.

In [11]:
# River network extraction
branches = grid.extract_river_network(fdir, acc > 50, dirmap=dirmap)

## Visualization of flow directions

**This part of the code shows the flow directions in the digital elevation model (DEM) along with the legend of the direction labels.**

- First, the flow direction labels are defined using the `dirmap_labels` list, which contains text labels for each direction.
- The color palette is set using the Seaborn library based on the `fiddle' palette.
- The graph is created using the function `plt.subplots(figsize=(8.5,6.5))`, where the X and Y axis sizes are set based on the scale of the Digital Elevation Model (DEM) grid.
- The plotting of each branch of the river network is done by iterating over all branches in the variable `branches['features']`.
- The legend is created manually using colors from the `violin` palette and direction labels from the `dirmap_labels` variable. The legend is added to the chart in the upper right corner with `plt.legend()`.

In [ ]:
# Flow direction labels list definition
dirmap_labels = ['East', 'Northeast', 'North', 'Northwest', 'West', 'Southwest', 'South', 'Southeast']

# Setting the palette colours using the Seaborn library
sns.set_palette('violin')

# Create map and axes
fig, ax = plt.subplots(figsize=(8.5,6.5))
plt.xlim(grid.bbox[0], grid.bbox[2])
plt.ylim(grid.bbox[1], grid.bbox[3])

# Set the same scale on both axes to maintain the aspect ratio
ax.set_aspect('equal')

# Iterate over each branch of the river grid
for branch in branches['features']:
    # Extracting branch line coordinates
    line = np.asarray(branch['geometry']['coordinates'])
    # Plotting the line
    plt.plot(line[:, 0], line[:, 1])

# Set the plot caption
_ = plt.title('Flow directions', size=14)

# List of colors from the violin palette
colors = sns.color_palette('violin', len(dirmap_labels))

# Manual creation of the legend
legend_handles = [mpatches.Patch(color=colors[i], label=dirmap_labels[i]) for i in range(len(dirmap_labels))]

# Adding a legend to the chart with the location in the upper right corner
plt.legend(handles=legend_handles, title='Flow directions', loc='upper right', fontsize='small')

## Calculating the distance from the mouth to each cell

**This section of code calculates the distance from the mouth to each cell in the digital terrain model.**

- The `grid.distance_to_outlet()` function calculates the distance from the outlet to each cell based on the specified parameters:
  - `x` and `y`: coordinates of the point on the surface that serves as the outlet
  - `fdir`: flow directions in the digital elevation model (DEM)
  - `dirmap`: map of directions
  - `xytype='coordinate'`: specification that the coordinates of the point are in geographic format.

- The resulting distance from the mouth to each cell is stored in the variable `dist`.

In [13]:
# Calculating the distance from the mouth to each cell
dist = grid.distance_to_outlet(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap,
                               xytype='coordinate')

## Flow distance visualization

**This part of the code visualizes the distance from the mouth to each cell in the digital terrain model**

- First, a figure and axis instance is created using the `plt.subplots(figsize=(10,6))` function. The transparency of the figure is set to zero.
- A grid is added using `plt.grid('on', zorder=0)` for better orientation.
- Distance rendering is done using `ax.imshow()`, where colormap 'cubehelix_r' is used to display the color scheme. Here `dist` contains the distances from the mouth to each cell.
- The colormap is added using `plt.colorbar()` with the label 'Distance to outflow (cell)'.
- The X and Y axis labels are set to 'Longitude' and 'Latitude' and the title is set to 'Distance to flow'.

In [ ]:
# Create visualization
fig, ax = plt.subplots(figsize=(10,6))
fig.patch.set_alpha(0)
plt.grid('on', zorder=0)

# Plot the distance from the mouth to each cell based on the DEM
im = ax.imshow(dist, extent=grid.extent, zorder=2,
               cmap='cubehelix_r')

# PLegenda and labels
plt.colorbar(im, ax=ax, label='Distance to drain (cell)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Distance to flow', size=14)